<a href="https://colab.research.google.com/github/Tomalison/hello-colab/blob/main/%E5%B0%88%E9%A1%8C%E5%AF%A6%E4%BD%9C_07_%E7%88%AC%E8%9F%B2%E8%B3%87%E6%96%99%E8%92%90%E9%9B%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import requests
import json


# 利用 requests 對 API 來源發送一個請求
url = https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
response = requests.get(url)

# 將請求回應的內容存成一個字串格式
d = response.text

# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(d)

print(data)